In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from scipy import stats
import statsmodels.api as sm

# Load and clean data
#def read_csv():
df = pd.read_csv('../input_data/combined_temperature.csv')
df.dropna()

,Country,Year,Annual Mean,5-yr smooth,Code
0,Dominican Republic,1901,23.71,23.65,DOM
1,Dominican Republic,1902,23.40,23.52,DOM
2,Dominican Republic,1903,23.45,23.38,DOM
3,Dominican Republic,1904,23.03,23.26,DOM
4,Dominican Republic,1905,23.44,23.16,DOM
...,...,...,...,...,...
20735,Switzerland,2018,7.68,7.27,CHE
20736,Switzerland,2019,7.31,7.34,CHE
20737,Switzerland,2020,7.64,7.38,CHE
20738,Switzerland,2021,6.39,7.42,CHE


In [ ]:
countries = df["Country"].unique() 

choice = input("Which countries graph would you like to see?: ") 
if choice.lower() == "all": 
    for country in countries: 
        country_data = df[df["Country"] == country] 
        plt.figure(figsize=(10, 5)) 
        sns.lineplot(data=country_data, x='Year', y='Annual Mean', hue='Country') 
        plt.title('Temperature Changes Over Time by Country') 
        plt.xlabel('Year')
        plt.ylabel('Annual Mean (°C)') 
        plt.legend(title='City', bbox_to_anchor=(1.05, 1), loc='upper left') 
        plt.show() 
        plt.savefig('../output/temperature_trends.png') 
        plt.close() 
else: 
    country_data = df[df["Country"] == choice] 
    plt.figure(figsize=(10, 5)) 
    sns.lineplot(data=country_data, x='Year', y='Annual Mean', hue='Country') 
    plt.title('Temperature Changes Over Time by Country') 
    plt.xlabel('Year') 
    plt.ylabel('Annual Mean (°C)') 
    plt.legend(title='City', bbox_to_anchor=(1.05, 1), loc='upper left') 
    plt.show() 
    plt.savefig('../output/temperature_trends.png') 
    plt.close()

In [ ]:
X = df['Year']
y = df['Annual Mean']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# Print regression results
print(model.summary())

In [ ]:
gdf = gpd.read_file('../input_data/countries.geo.json')
gdf = gdf.merge(df, left_on='name',right_on='Country')

fig, ax = plt.subplots(figsize=(10, 5))
gdf.plot(column='Annual Mean', ax=ax, legend=True, 
         legend_kwds={'label': 'Annual Mean', 'orientation': 'horizontal'},
         cmap='YlGn')
ax.set_title('Annual Mean')
plt.axis('off')
plt.tight_layout()
plt.show()
plt.savefig('../output/annual_mean_map.png')
plt.close()

# Assuming df is your DataFrame with 'Country' and 'Annual Mean' columns
df['Temperature Change'] = df.groupby('Country')['Annual Mean'].transform(lambda x: x.max() - x.min())

gdf = gpd.read_file('../input_data/countries.geo.json')
gdf = gdf.merge(df, left_on='name', right_on='Country')

fig, ax = plt.subplots(figsize=(10, 5))
gdf.plot(column='Temperature Change', ax=ax, legend=True, 
         legend_kwds={'label': 'Change in Temperature (°C)', 'orientation': 'horizontal'},
         cmap='RdBu_r')  # RdBu_r is a diverging colormap (blue for cooling, red for warming)
ax.set_title('World view of Temperature Change (max - min)')
plt.axis('off')
plt.tight_layout()
plt.show()
plt.savefig('../output/annual_mean_maxminusmin_map.png')
plt.close()

In [ ]:
# Calculate correlation between temperature and green space size
csv_path = input("Enter the path to the csv file you want to compare: ")
country_name = input("Enter the column name in your csv file the represents the country: ")
value_to_compare = input("Enter the column name in you csv file that represents the value you want to compare: ")

compare_df = pd.read_csv(csv_path)
compare_df.dropna()
temp_change = df.groupby('Country').agg({
    'Year': ['min', 'max'],
    'Annual Mean': ['first', 'last']
}).reset_index()

temp_change.columns = [country_name, 'First_Year', 'Last_Year', 'First_Temp', 'Last_Temp']
temp_change['Temperature_Change'] = temp_change['Last_Temp'] - temp_change['First_Temp']

total_emissions = compare_df.groupby(country_name)[value_to_compare].sum().reset_index()

merged_data = pd.merge(temp_change, total_emissions, on=country_name)

correlation = merged_data['Temperature_Change'].corr(merged_data[value_to_compare])
print(f"Correlation between Temperature Change and Total CO2 Emissions: {correlation:.2f}")

In [ ]:
# Perform t-test to assess statistical significance
t_stat, p_value = stats.ttest_ind(merged_data['Temperature_Change'], merged_data[value_to_compare])
print(f"T-statistic: {t_stat:.2f}")
print(f"P-value: {p_value:.4f}")